In [93]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [95]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [98]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [99]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [100]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [101]:
str_dict ='[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dick = {}
  if str(x) == 'nan': return output_dick

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dick[key] = value
  return output_dick

df['features_parsed'] = df['features'].map(parse_features)

In [103]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [104]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)


In [105]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_����������������', 'feat_outer material', 'feat_kids backpacks',
       'feat_assembled in country of origin', 'feat_style code',
       'feat_fastener', 'feat_metal color', 'feat_bridge',
       'feat_handbag style', 'feat_size/dimensions'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [107]:
{ k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+ '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [109]:
df['brand'] = df['brand'].map(lambda x:str(x).lower() )
df[df.brand == df.feat_brand ].shape

(8846, 1002)

In [0]:
feats = ['']

In [111]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.36192388096632, 4.184062936250326)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]
#feats_cat

In [0]:
feats = ['brand_cat', 
         'feat_fabric content_cat',       
         'feat_adjustable_cat' , 
         'feat_case diameter_cat',
         'feat_shoe category_cat',         
         'feat_brand_cat', 
         'feat_material_cat']


model = RandomForestRegressor(max_depth=100, n_estimators=100)
result = run_model(feats, model)

In [0]:
X = df[ feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=100, n_estimators=100, random_state=100)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-45.97860967834497, 3.7587685162632827)


In [115]:
df['brand'].value_counts(normalize=True)

nike               0.097210
puma               0.033315
ralph lauren       0.028775
vans               0.021116
new balance        0.020295
                     ...   
econoled           0.000055
j.d. fisk          0.000055
andrew stevens     0.000055
electric visual    0.000055
mo&y               0.000055
Name: brand, Length: 1732, dtype: float64

In [116]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{}, {'sport': 'football', 'condition': 'new with box'},
       {'manufacturer part number': '342964-033', 'gender': 'men', 'shoe category': 'athletic & sneakers', 'brand': 'nike', 'color': 'silver'},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '11.5', 'size': '11.5', 'color': 'green', 'manufacturer part number': '652775 303', 'brand': 'nike', 'age group': 'adult'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'calypso/white', 'casual & dress shoe style': 'athletic sneakers', 'manufacturer part number': '718552410', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [117]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
mens                   4
infant                 4
toddler                4
boys'                  3
youth                  2
women ,�� unisex       2
women                  2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [0]:
!git add matrix_one/Day5.ipynb

In [119]:
!git commit -m 'Dzien 5 '

[master e064592] Dzien 5
 1 file changed, 1 insertion(+), 1 deletion(-)


In [0]:
!git config --global user.email "paulinaxbanas@gmail.com"
!git config --global user.name "Paulina"

In [121]:
!git push -u origin master

Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.43 KiB | 486.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/xpandax2306/dw_matrix.git
   b010744..e064592  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.
